In [11]:
import math

def emo_lev(arousal,valence):    #Function that returns emotion and intensity
    arousal -= 5
    valence -= 5
    r = 5
    x, y = valence, arousal
    dist = (x**2+y**2)**0.5

    #Calc theta in radian
    theta = math.atan(y/x)

    #Convert theta from rad to deg
    theta = 180 * theta/math.pi

    def lev(dist):
        if(dist<1.67):
            return "Low"
        if(dist>1.67 and dist<3.34):
            return "Medium"
        else:
            return "High"
        
    #Check which intensity
    levl = lev(dist)


    def conv_emo(theta):
        if(x>0 and y>0):  #1st Quadrant
            if theta <= 90:
                if(theta<45):
                    return "Happy",1.1,theta  #1st Quad, 1st half
                else:
                    return "Excited",1.2,theta #1st Quad, 2nd half
        else:
            if(x>0 and y<0): #4
                theta+=360
                quad = 4
                if theta <= 360:
                    if(theta<315):
                        return "Calm",4.1,theta
                    else:
                        return "Content",4.2,theta
            else:
                if(x<0 and y>0): #2
                    theta+=180
                    quad = 2
                    if theta <= 180:
                        if(theta<135):
                            return "Angry",2.1,theta
                        else:
                            return "Afraid",2.2,theta
                else:
                    if(x<0 and y<0):
                       theta+=180
                       if theta <= 270:
                            if(theta<225):
                                return "Sad",3.1,theta
                            else:
                                return "Depressed",3.2,theta
                       
                   
    #print("Theta before conversion: {}".format(theta))
    emo,quad,theta = conv_emo(theta)
    #print("Distance = {} \nTheta after conversion = {} \nquadrant = {} ".format(dist,theta,quad))
    return emo,levl,theta,quad    #END of emo_lev

#Input the valence, arousal values here.
valence_x, arousal_y = 2,3
emotion, level, theta,quadrant = emo_lev(arousal_y,valence_x)

#Optional printing. Just return the values you need if you want.
print("Emotion    : "+emotion)
print("Intensity  : "+level)
print("Theta      : {} degree".format("%.2f"%theta))
print("Quadrant   : {}".format(quadrant))

Emotion    : Sad
Intensity  : High
Theta      : 213.69 degree
Quadrant   : 3.1


In [16]:
from models.tcn import *
from dataset.DEAP_DATASET import ModularDeapDataset
from torch.utils.data import DataLoader
import torch
from util.train import select_last_elm

DATA_SET_PATH = 'dataset/'
deap_val = ModularDeapDataset(DATA_SET_PATH, train=False)
deap_train = ModularDeapDataset(DATA_SET_PATH, train=True)

train_loader = DataLoader(deap_train, shuffle=True, batch_size=1)
val_loader = DataLoader(deap_val, shuffle=True, batch_size=1)

CHAN_LIST = [150, 64, 2]  # The list of each convolutional layers
KERN_SIZE = 5
DROP_OUT = 0.2

MDL_PATH = 'models/saved_weights/tcn_v0.pth'

model = EEG_TCN(CHAN_LIST, KERN_SIZE, DROP_OUT)

model.load_state_dict(torch.load(MDL_PATH, map_location="cpu"))
model.eval()
with torch.no_grad():
    for i, (signal, label) in enumerate(train_loader):

        output = None

        output = model(signal)
        output = select_last_elm(output, "cpu")
        output = output.numpy()
        label = label.numpy()
        
        print("Predicted:", output, "Actual Label:", label)
        
        emotion, level, theta,quadrant = emo_lev(output[0][1],output[0][0])
        print("PREDICTED",output)
        print("\tEmotion    : "+emotion)
        print("\tIntensity  : "+level)
        print("\tTheta      : {} degree".format("%.2f"%theta))
        print("\tQuadrant   : {}".format(quadrant))
        
        emotion, level, theta,quadrant = emo_lev(label[0][1],label[0][0])
        print("ACTUAL",label)
        print("\tEmotion    : "+emotion)
        print("\tIntensity  : "+level)
        print("\tTheta      : {} degree".format("%.2f"%theta))
        print("\tQuadrant   : {}".format(quadrant))


Predicted: [[5.68231  5.485541]] Actual Label: [[7.12 7.09]]
PREDICTED [[5.68231  5.485541]]
	Emotion    : Happy
	Intensity  : Low
	Theta      : 35.44 degree
	Quadrant   : 1.1
ACTUAL [[7.12 7.09]]
	Emotion    : Happy
	Intensity  : Medium
	Theta      : 44.59 degree
	Quadrant   : 1.1
Predicted: [[5.9923573 5.7626014]] Actual Label: [[4.06 7.05]]
PREDICTED [[5.9923573 5.7626014]]
	Emotion    : Happy
	Intensity  : Low
	Theta      : 37.54 degree
	Quadrant   : 1.1
ACTUAL [[4.06 7.05]]
	Emotion    : Angry
	Intensity  : Medium
	Theta      : 114.63 degree
	Quadrant   : 2.1
Predicted: [[4.9677477 4.8784366]] Actual Label: [[4.58 4.96]]
PREDICTED [[4.9677477 4.8784366]]
	Emotion    : Depressed
	Intensity  : Low
	Theta      : 255.14 degree
	Quadrant   : 3.2
ACTUAL [[4.58 4.96]]
	Emotion    : Sad
	Intensity  : Low
	Theta      : 185.44 degree
	Quadrant   : 3.1
Predicted: [[4.5700355 4.5535507]] Actual Label: [[4.   5.04]]
PREDICTED [[4.5700355 4.5535507]]
	Emotion    : Depressed
	Intensity  : Low
	T

/home/adam/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


TypeError: cannot unpack non-iterable NoneType object